# 로컬에서 훈련 하기
- https://www.kaggle.com/code/mitanshuchakrawarty/fine-tune-llm-for-text-summary

## 1. 환경 셋업

In [8]:
from dotenv import load_dotenv

import os

HF_TOKEN = os.getenv('HF_TOKEN')

!huggingface-cli login --token {HF_TOKEN}

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/ec2-user/SageMaker/.cache/token
Login successful


In [9]:
import os 
os.environ['TRANSFORMERS_CACHE'] = "/home/ec2-user/SageMaker/.cache" 
os.environ['HF_DATASETS_CACHE'] = "/home/ec2-user/SageMaker/.cache" 
os.environ['HF_HOME'] = "/home/ec2-user/SageMaker/.cache"

In [10]:
%store -r data_folder
%store -r train_data_json 
%store -r validation_data_json 
%store -r test_data_json 

print("data_folder: ", data_folder)
print("train_data_json: ", train_data_json)
print("validation_data_json: ", validation_data_json)
print("test_data_json: ", test_data_json)

data_folder:  ../data/naver-news-summarization-ko
train_data_json:  ../data/naver-news-summarization-ko/train_dataset.json
validation_data_json:  ../data/naver-news-summarization-ko/validation_dataset.json
test_data_json:  ../data/naver-news-summarization-ko/test_dataset.json


In [11]:
import torch
import time
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

from datasets import Dataset, load_dataset
from datasets import load_dataset, load_metric
from transformers import pipeline, set_seed
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import warnings
warnings.filterwarnings("ignore")

## 2. 베이스 모델 준비

In [12]:
model_id = "meta-llama/Meta-Llama-3-8B"
output_dir = "/home/ec2-user/SageMaker/models/llama-3-8b-naver-news"

### Config YAML 파일 생성

In [13]:
%%writefile local_llama_3_8b_fsdp_qlora.yaml
# script parameters
model_id:  "meta-llama/Meta-Llama-3-8B" # Hugging Face model id
dataset_path: "../data/naver-news-summarization-ko"                      # path to dataset
max_seq_len:  2048              # max sequence length for model and packing of the dataset
# training parameters
output_dir: "/home/ec2-user/SageMaker/models/llama-3-8b-naver-news" # Temporary output directory for model checkpoints
report_to: "tensorboard"               # report metrics to tensorboard
learning_rate: 0.0002                  # learning rate 2e-4
lr_scheduler_type: "constant"          # learning rate scheduler
num_train_epochs: 1                    # number of training epochs
per_device_train_batch_size: 1         # batch size per device during training
per_device_eval_batch_size: 1          # batch size for evaluation
gradient_accumulation_steps: 2         # number of steps before performing a backward/update pass
optim: adamw_torch                     # use torch adamw optimizer
logging_steps: 10                      # log every 10 steps
save_strategy: epoch                   # save checkpoint every epoch
evaluation_strategy: epoch             # evaluate every epoch
max_grad_norm: 0.3                     # max gradient norm
warmup_ratio: 0.03                     # warmup ratio
bf16: true                             # use bfloat16 precision
tf32: true                             # use tf32 precision
gradient_checkpointing: true           # use gradient checkpointing to save memory
# FSDP parameters: https://huggingface.co/docs/transformers/main/en/fsdp
fsdp: "full_shard auto_wrap offload" # remove offload if enough GPU memory
fsdp_config:
  backward_prefetch: "backward_pre"
  forward_prefetch: "false"
  use_orig_params: "false"

Overwriting local_llama_3_8b_fsdp_qlora.yaml


## 3. 훈련 Script 실행

In [14]:
!ACCELERATE_USE_FSDP=1 FSDP_CPU_RAM_EFFICIENT_LOADING=1 torchrun --nproc_per_node=4 \
../scripts/local_run_fsdp_qlora.py \
--config local_llama_3_8b_fsdp_qlora.yaml

[2024-06-29 07:56:46,632] torch.distributed.run: [WARNING] 
[2024-06-29 07:56:46,632] torch.distributed.run: [WARNING] *****************************************
[2024-06-29 07:56:46,632] torch.distributed.run: [WARNING] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
[2024-06-29 07:56:46,632] torch.distributed.run: [WARNING] *****************************************
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  

## 4. 베이스 모델과 훈련된 모델 머지

In [15]:
model_id, output_dir

('meta-llama/Meta-Llama-3-8B',
 '/home/ec2-user/SageMaker/models/llama-3-8b-naver-news')

### 모델 머지 및 로컬에 저장

In [16]:
from peft import AutoPeftModelForCausalLM
import torch

# Load PEFT model on CPU

model = AutoPeftModelForCausalLM.from_pretrained(
    output_dir,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)  
# Merge LoRA and base model and save
merged_model = model.merge_and_unload()
merged_model.save_pretrained(output_dir,safe_serialization=True, max_shard_size="2GB")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### 머지된 모델 로딩

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer 


# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  pretrained_model_name_or_path = output_dir,
  torch_dtype=torch.float16,
  quantization_config= {"load_in_4bit": True},
  device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(output_dir)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 5. 추론

### 테스트 데이터 셋 로딩

In [ ]:
test_data_json

'../data/naver-news-summarization-ko/test_dataset.json'

In [ ]:
from datasets import load_dataset 
from random import randint

# Load our test dataset
eval_dataset = load_dataset("json", data_files=test_data_json, split="train")
rand_idx = randint(0, len(eval_dataset))
messages = eval_dataset[rand_idx]["messages"][:2]
rand_idx, messages

(8,
 [{'content': 'You are an AI assistant specialized in news articles.Your role is to provide accurate summaries and insights in Korean. Please analyze the given text and provide concise, informative summaries that highlight the key goals and findings.',
   'role': 'system'},
  {'content': 'Please summarize the goals for journalist in this text:\n\n아이폰13프로맥스 120만원대에서 150만원대로 지난달 아이패드 재책정 이어 아이폰 가격도 올라 디지털데일리 백승은 기자 애플이 일본에서 아이폰 일부 모델 가격을 인상했다. 최근 엔화 가치가 크게 떨어지자 내린 조치다. 30일 애플 일본 웹사이트에 따르면 아이폰SE 아이폰13미니 아이폰13 아이폰13프로 아이폰13프로맥스 가격이 9 19% 상승했다. 가장 저렴한 아이폰SE의 경우 5만7800엔 약 55만원 에서 6만2800엔 약 59만원 으로 올랐다. 가장 고가 제품인 아이폰13프로맥스는 13만4800엔 약 128만원 에서 15만9800엔 약 152만원 으로 재책정됐다. 애플은 지난 6월 일본에서 판매 중인 아이패드 가격도 올렸다. 대상 제품은 아이패드 아이패드에어 아이패드프로 11인치 아이패드프로 12.9인치 아이패드미니다. 인상률이 가장 높은 제품은 아이패드로 기존 3만9800엔 약 37만원 에서 25% 오른 4만9800엔 약 47만원 이 됐다. 애플의 가격 인상 조치는 올해 초부터 나타난 엔저 효과에서 기인한 것으로 보인다. 엔저 효과란 국제 환시세에서 엔 값이 타국 화폐에 비해 낮아지는 현상을 말한다. 환율이 약세하면 수입물가가 오를 가능성이 있다. 이에 대비하기 위해 가격을 올린 것으로 분석된다. 한편 애플은 일본 스마트폰 시장에서

### 추론

In [ ]:
# Test on sample 
input_ids = tokenizer.apply_chat_template(messages,add_generation_prompt=True,return_tensors="pt").to(model.device)
outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id= tokenizer.eos_token_id,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]

print(f"**Query:**\n{eval_dataset[rand_idx]['messages'][1]['content']}\n")
print(f"**Original Answer:**\n{eval_dataset[rand_idx]['messages'][2]['content']}\n")
print(f"**Generated Answer:**\n{tokenizer.decode(response,skip_special_tokens=True)}")



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


**Query:**
Please summarize the goals for journalist in this text:

아이폰13프로맥스 120만원대에서 150만원대로 지난달 아이패드 재책정 이어 아이폰 가격도 올라 디지털데일리 백승은 기자 애플이 일본에서 아이폰 일부 모델 가격을 인상했다. 최근 엔화 가치가 크게 떨어지자 내린 조치다. 30일 애플 일본 웹사이트에 따르면 아이폰SE 아이폰13미니 아이폰13 아이폰13프로 아이폰13프로맥스 가격이 9 19% 상승했다. 가장 저렴한 아이폰SE의 경우 5만7800엔 약 55만원 에서 6만2800엔 약 59만원 으로 올랐다. 가장 고가 제품인 아이폰13프로맥스는 13만4800엔 약 128만원 에서 15만9800엔 약 152만원 으로 재책정됐다. 애플은 지난 6월 일본에서 판매 중인 아이패드 가격도 올렸다. 대상 제품은 아이패드 아이패드에어 아이패드프로 11인치 아이패드프로 12.9인치 아이패드미니다. 인상률이 가장 높은 제품은 아이패드로 기존 3만9800엔 약 37만원 에서 25% 오른 4만9800엔 약 47만원 이 됐다. 애플의 가격 인상 조치는 올해 초부터 나타난 엔저 효과에서 기인한 것으로 보인다. 엔저 효과란 국제 환시세에서 엔 값이 타국 화폐에 비해 낮아지는 현상을 말한다. 환율이 약세하면 수입물가가 오를 가능성이 있다. 이에 대비하기 위해 가격을 올린 것으로 분석된다. 한편 애플은 일본 스마트폰 시장에서 과반 이상 점유율을 차지하고 있다. 시장조사업체 스트래티지애널리틱스 SA 에 따르면 애플은 지난 1분기 일본 스마트폰 시장에서 56.8%를 기록하며 압도적인 1위를 기록했다.

**Original Answer:**
애플이 일본에서 아이폰 일부 모델 가격을 인상하고 지난 6월 일본에서 판매 중인 아이패드 가격도 올린 가운데, 시장조사업체 스트래티지애널리틱스 SA 에 따르면 지난 1분기 일본 스마트폰 시장에서 압도적인 1위를 기록하며 압도적인 1위를 기록했다.

**Generated Answer:**
1. 